## Final Project Submission

Please fill out:
* Student name: Calvin Omwega
* Student pace: Full time
* Scheduled project review date/time: N/A
* Instructor name: Maryann Mwikali
* Blog post URL:N/A


## 1.Introduction

# 1.1 Overview
The project's primary purpose is to generate insights that can assist Microsoft in entering and succeeding in the movie industry. We will utilize exploratory data analysis to get actionable insights to assist Microsoft determine what kind of films to develop. This project will leverage existing movie data to analyze box office trends, genre performance, and financial metrics to recommend strategic directions for Microsoft's new movie studio.

# 1.2 General Objectives
1. Identify Successful Film Genres: Determine which movie genres are performing best at the box office, both domestically and internationally.

2. Analyze Budgets versus Returns: Examine the relationship between film budgets and their financial success to identify optimal investment levels.

3. Understand Market Trends: Capture current trends in movie consumption and preferences, including the impact of digital and streaming platforms.

4. Provide Strategic Recommendations: Translate findings into actionable strategies for genre selection, budget allocation, and technological integration for Microsoft's studio.

# 1.3 Key Questions 
1. Which movie genres yield the highest box office earnings and ROI?
Aims to identify genres that consistently perform well, offering a safer investment for Microsoft.
2. What is the optimal budget range for producing a profitable movie?
Seeks to find a budget sweet spot that maximizes earnings without unnecessarily inflating costs.
3. How do movie budgets correlate with box office success?
Investigates whether higher budgets lead to higher gross earnings or if diminishing returns set in beyond a certain point.

# 2. Data Sets
The data sets are stored in the 'zippedData' folder which were sourced from:

- Box Office MojoLinks
    - bom.movie_gross.csv
- TheMovieDBLinks
    - tmdb.movies.csv
- The Numbers
    - tn.movie_budgets.csv


# 2.2 Iporting the libraries

In [5]:
# Your code here - remember to use markdown cells for comments as well!
# importing libraries
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 

%matplotlib inline

In [8]:
# Loading the datasets
df_gross = pd.read_csv('bom.movie_gross.csv')
df_tmdb = pd.read_csv('tmdb.movies.csv')
df_budgets = pd.read_csv('tn.movie_budgets.csv')

In [21]:
# Displaying the first few rows of the dataset to understand its structure
print(df_gross.head())
df_gross.shape

                                         title studio  domestic_gross  \
0                                  Toy Story 3     BV     415000000.0   
1                   Alice in Wonderland (2010)     BV     334200000.0   
2  Harry Potter and the Deathly Hallows Part 1     WB     296000000.0   
3                                    Inception     WB     292600000.0   
4                          Shrek Forever After   P/DW     238700000.0   

   foreign_gross  year  
0    652000000.0  2010  
1    691300000.0  2010  
2    664300000.0  2010  
3    535700000.0  2010  
4    513900000.0  2010  


(3387, 5)

In [22]:
# This is still displaying the data sets and the first few rows of the datasets to understand its structure
print(df_tmdb.head())
df_tmdb.shape

   Unnamed: 0            genre_ids     id original_language  \
0           0      [12, 14, 10751]  12444                en   
1           1  [14, 12, 16, 10751]  10191                en   
2           2        [12, 28, 878]  10138                en   
3           3      [16, 35, 10751]    862                en   
4           4        [28, 878, 12]  27205                en   

                                 original_title  popularity release_date  \
0  Harry Potter and the Deathly Hallows: Part 1      33.533   2010-11-19   
1                      How to Train Your Dragon      28.734   2010-03-26   
2                                    Iron Man 2      28.515   2010-05-07   
3                                     Toy Story      28.005   1995-11-22   
4                                     Inception      27.920   2010-07-16   

                                          title  vote_average  vote_count  
0  Harry Potter and the Deathly Hallows: Part 1           7.7       10788  
1           

(26517, 10)

In [24]:
# This is the last display of the last data set being used
print(df_budgets.head())
df_budgets.shape

   id  release_date                                        movie  \
0   1  Dec 18, 2009                                       Avatar   
1   2  May 20, 2011  Pirates of the Caribbean: On Stranger Tides   
2   3   Jun 7, 2019                                 Dark Phoenix   
3   4   May 1, 2015                      Avengers: Age of Ultron   
4   5  Dec 15, 2017            Star Wars Ep. VIII: The Last Jedi   

   production_budget  domestic_gross  worldwide_gross  
0          425000000       760507625       2776345279  
1          410600000       241063875       1045663875  
2          350000000        42762350        149762350  
3          330600000       459005868       1403013963  
4          317000000       620181382       1316721747  


(5782, 6)

In [19]:
print(df_gross['title'].dtype)
print(df_budgets['movie'].dtype)


object
object


In [20]:
df_gross['title'] = df_gross['title'].astype(str)
df_budgets['movie'] = df_budgets['movie'].astype(str)
